In [2]:
!pip install openai

zsh:1: /Users/dp/Projects/launching-agent/WebApp/.venv/bin/pip: bad interpreter: /Users/dp/Projects/launching-agent/frame/.venv/bin/python: no such file or directory
Looking in indexes: https://repo.mlops.dp.tech/repository/pypi-group/simple


In [2]:
import json
import time
from pydantic import BaseModel
from openai import OpenAI

In [3]:
openai = OpenAI(
    base_url="http://localhost:3000",
    api_key="EMPTY"
)

def model_to_json_str(model: BaseModel):
    return json.dumps(model.model_dump(), indent=4)

def get_current_weather(location):
    return { "temperature": "68", "unit": "F" }

In [7]:
def create_assistant():
    assistant = openai.beta.assistants.create(
        instructions="You are a weather bot. Use the provided functions to answer questions.",
        model="gpt-4",
        name="Weather Bot",
        tools=[{
            "type": "function",
            "function": {
                "name": "getCurrentWeather",
                "description": "Get the weather in location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": { "type": "string", "description": "The city and state e.g. San Francisco, CA" },
                        "unit": { "type": "string" }
                    },
                    "required": ["location"]
                }
            }
        }]
    )
    print("CREATING ASSISTANT: ", model_to_json_str(assistant))
    return assistant

def create_thread():
    thread = openai.beta.threads.create()
    print("CREATING THREAD: ", model_to_json_str(thread))
    return thread

def create_message(thread_id):
    message = openai.beta.threads.messages.create(
        thread_id,
        role="user",
        content="What's the weather in San Francisco?"
    )
    print("CREATING MESSAGE: ", model_to_json_str(message))
    return message

def create_run(thread_id, assistant_id):
    run = openai.beta.threads.runs.create(
        thread_id,
        assistant_id=assistant_id,
        instructions="You are a weather bot. Use the provided functions to answer questions."
    )
    print("CREATING RUN: ", model_to_json_str(run))
    return run

def get_run(thread_id, run_id):
    run = openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
    print("GETTING RUN: ", model_to_json_str(run))
    return run

def submit_tool_outputs(thread_id, run_id, tool_call_id, args):
    weather = get_current_weather(args['location'])
    run = openai.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=[
            {
                "tool_call_id": tool_call_id,
                "output": json.dumps(weather)
            }
        ]
    )
    print("SUBMIT TOOL OUTPUTS: ", model_to_json_str(run))
    return run

def get_messages(thread_id):
    messages = openai.beta.threads.messages.list(thread_id)
    print("GETTING MESSAGES: ", model_to_json_str(messages))
    return messages

def main():
    assistant = create_assistant()
    thread = create_thread()
    thread_id = thread.id
    create_message(thread_id)
    run = create_run(thread_id, assistant.id)

    while True:
        run_status = get_run(thread_id, run.id)
        if run_status.status == 'requires_action':
            tool_call = run_status.required_action.submit_tool_outputs.tool_calls[0]
            tool_call_json_args = json.loads(tool_call.function.arguments)
            submit_tool_outputs(thread_id, run.id, tool_call.id, tool_call_json_args)
            break
        time.sleep(1)


    while True:
        messages = get_messages(thread_id)
        print("MESSAGES: ", model_to_json_str(messages))
        if any(message.role == 'assistant' for message in messages.data):
            break
        time.sleep(1)

In [8]:
main()

CREATING ASSISTANT:  {
    "id": "b263a868-7306-4494-b21a-851782257853",
    "created_at": 1708507554,
    "description": null,
    "file_ids": [],
    "instructions": "You are a weather bot. Use the provided functions to answer questions.",
    "metadata": null,
    "model": "gpt-4",
    "name": "Weather Bot",
    "object": "",
    "tools": [
        {
            "function": {
                "name": "getCurrentWeather",
                "parameters": {
                    "properties": {
                        "location": {
                            "description": "The city and state e.g. San Francisco, CA",
                            "type": "string"
                        },
                        "unit": {
                            "type": "string"
                        }
                    },
                    "required": [
                        "location"
                    ],
                    "type": "object"
                },
                "description": 

KeyboardInterrupt: 